# Train model

In [16]:
!nvidia-smi

Sat Apr 10 16:28:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 455.32.00    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:00:05.0 Off |                  N/A |
| 32%   29C    P8     4W / 250W |      1MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  On   | 00000000:00:06.0 Off |                  N/A |
| 31%   

Set environment variables

In [18]:
#os.environ["train_path"] = train_path
#os.environ["eval_path"] = eval_path
os.environ["CUDA_LAUNCH_BLOCKING"]='1'  #Makes for easier debugging (just in case)
weights_dir = "/home/leonardovida/dev/hist-aware/notebooks/models/bert-training-from-scratch/weights"
#!mkdir {weights_dir}

In [19]:
torch.cuda.is_available()

True

In [20]:
config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=512,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1, # The vocabulary size of the token_type_ids passed
)

Paper BERTje
--max_predictions_per_seq=20 \
  --train_batch_size=256 \
  --eval_batch_size=32 \
  --learning_rate=1e-4 \
  --num_train_steps=1000000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --iterations_per_loop=10000 \
  --max_eval_steps=10000 \

In [27]:
tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_dir, max_len=512)

In [28]:
model = RobertaForMaskedLM(config=config)

In [23]:
model.num_parameters()
# => 84 million parameters

83502880

### Create Dataset

In [4]:
from torch.utils.data import Dataset

class DelphDataset(Dataset):
    def __init__(self, evaluate: bool = False):
        tokenizer = ByteLevelBPETokenizer(
            tokenizer_dir + "/vocab.json",
            tokenizer_dir + "/merges.txt",
        )
        tokenizer._tokenizer.post_processor = BertProcessing(
            ("</s>", tokenizer.token_to_id("</s>")),
            ("<s>", tokenizer.token_to_id("<s>")),
        )
        tokenizer.enable_truncation(max_length=512)
        # or use the RobertaTokenizer from `transformers` directly.

        self.examples = []

        #src_files = Path(PATH_RAW_FILES).glob("*-eval.txt") if evaluate else Path(PATH_RAW_FILES).glob("*-train.txt")
        src_files = Path(PATH_RAW_FILES).glob("*.txt")
        for src_file in src_files:
            print("🔥", src_file)
            lines = src_file.read_text(encoding="utf-8").splitlines()
            self.examples += [x.ids for x in tokenizer.encode_batch(lines)]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return torch.tensor(self.examples[i])

In [5]:
dataset = DelphDataset()

NameError: name 'tokenizer_dir' is not defined

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./oscar.eo.txt",
    block_size=128,
)

Test

In [43]:
files = [str(x) for x in Path(PATH_RAW_FILES).glob("*.txt")]

from datasets import load_dataset
from transformers import PreTrainedTokenizerFast

dataset = load_dataset('text', data_files=files, split='train')

# Load it using transformers
tokenizer = PreTrainedTokenizerFast(tokenizer_file=f"{tokenizer_dir}/byte-level-BPE.tokenizer.json")

def encode(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding='max_length',
        # TODO: padding 
        max_length=512,
    )

dataset = dataset.map(encode, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

dataloader = torch.utils.data.DataLoader(dataset, batch_size=4)
next(iter(dataloader))

Using custom data configuration default-4720c1f9f258d4c1
Reusing dataset text (/home/leonardovida/.cache/huggingface/datasets/text/default-4720c1f9f258d4c1/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)
Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


RuntimeError: stack expects each tensor to be equal size, but got [202] at entry 0 and [157] at entry 1

In [37]:
def prepare_data(dataset_f: str,
                 tokenizer: PreTrainedTokenizer,
                 batch_size: int = 16,
                 num_workers: int = 2) -> Dict[str, DataLoader]:
    """Given an input file, prepare the train, test, validation dataloaders.
       The created datasets will be preprocessed and save to disk.
    :param dataset_f: input file
    :param tokenizer: pretrained tokenizer that will prepare the data, i.e. convert tokens into IDs
    :param batch_size: batch size for the dataloaders
    :param num_workers: number of CPU workers to use during dataloading. On Windows this must be zero
    :return: a dictionary containing train, test, validation dataloaders
    """

    def collate(batch: List[Dict[str, Tensor]]) -> Dict[str, Tensor]:
        """Collates gathered items to form a batch which is then used in training, evaluation, or testing.
        :param batch: a list of samples from the dataset. Each sample is a dictionary with keys "input_ids".
        :return: the created batch with keys "input_ids"
        """
        all_input_ids = pad_sequence([item["input_ids"] for item in batch]).to(torch.long)

        return {"input_ids": all_input_ids}

    def preprocess(sentences: List[str]) -> Dict[str, Union[list, Tensor]]:
        """Preprocess the raw input sentences from the text file.
        :param sentences: a list of sentences (strings)
        :return: a dictionary of "input_ids"
        """
        tokens = [s.split() for s in sentences]

        # The sequences are not padded here. we leave that to the dataloader in collate
        # That means: a bit slower processing, but a smaller saved dataset size
        return tokenizer(tokens,
                         add_special_tokens=False,
                         return_token_type_ids=False,
                         return_attention_mask=False
                        )
    
    dataset = Dataset.from_dict({"text": Path(dataset_f).read_text(encoding="utf-8").splitlines()})

    # Split the dataset into train, test, dev
    # 90% (train), 10% (test + validation)
    train_testvalid = dataset.train_test_split(test_size=0.1)
    # 10% of total (test), 10% of total (validation)
    test_valid = train_testvalid["test"].train_test_split(test_size=0.5)

    dataset = DatasetDict({"train": train_testvalid["train"],
                           "test": test_valid["test"],
                           "valid": test_valid["train"]})

    dataset = dataset.map(preprocess, input_columns=["text"], batched=True)
    dataset.set_format("torch", columns=["input_ids"])

    return {partition: DataLoader(ds,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  collate_fn=collate,
                                  num_workers=num_workers,
                                  pin_memory=True) for partition, ds in dataset.items()}

In [38]:
prepare_data(dataset_f = "/home/leonardovida/data-histaware/articles0_50000.txt",
    tokenizer = tokenizer,
    batch_size = 16,
    num_workers = 2)

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

### Create Data Collator

To make backpropagation on

In [25]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

### Initalize Trainer

In [26]:
training_args = TrainingArguments(
    output_dir="home/leonardovida/data-histaware/delphBERT",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

NameError: name 'dataset' is not defined

In [ ]:
%%time
trainer.train()

#### Save model

In [ ]:
trainer.save_model("./EsperBERTo")

### Verify learning

Create a fill-mask pipeline to check whether the model learned anything useful

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./EsperBERTo",
    tokenizer="./EsperBERTo"
)

In [ ]:
fill_mask("La suno <mask>.")

### Upload the model to HuggingFace

Write a README.md model card and add it to the repository under `model_cards/`. Your model card should ideally include:
* a model description,
* training params (dataset, preprocessing, hyperparameters), 
* evaluation results,
* intended uses & limitations
* whatever else is helpful! 🤓

In [ ]:
#transformers-cli upload